In [301]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import os
import re
import json
import optuna
import ast
import matplotlib.pyplot as plt
import random
from matplotlib.backends.backend_pdf import PdfPages

In [302]:
path = "/Users/amrtamer/Documents/Internship"
# path = "/content/drive/MyDrive/Internship"

In [303]:
data = pd.read_csv(path + r'/data/Volve/Volve_cleaned_prepared.csv')
if "Unnamed: 0" in data.columns:
    data = data.drop(columns="Unnamed: 0")
data.head()

,well_name,WELL_BORE_CODE,date,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,...,linear_regressor,EMA_short,EMA_medium,EMA_long,Rolling_short,Rolling_medium,Rolling_long,Lag_short,Lag_medium,Lag_long
0,7405,NO 15/9-F-1 C,2014-04-22,0.268405,1.029309,0.605973,0.396165,-2.015211,-0.533280,3.281836,...,-0.489929,-0.495877,-0.498713,-0.502834,-0.001433,-0.0027,-0.005271,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,7405,NO 15/9-F-1 C,2014-04-23,0.268405,0.845793,0.634777,0.239816,0.088574,-0.419656,2.849981,...,-0.489929,-0.363747,-0.426301,-0.464667,-0.001433,-0.0027,-0.005271,"[0.0, 0.0, -0.4905010756171413]","[0.0, 0.0, 0.0, 0.0, -0.4905010756171413]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7405,NO 15/9-F-1 C,2014-04-24,0.268405,0.775023,0.639809,0.199889,-0.687843,-0.402874,2.607708,...,-0.146007,-0.180982,-0.315168,-0.402787,-0.001433,-0.0027,-0.005271,"[0.0, -0.4905010756171413, -0.09810136648612545]","[0.0, 0.0, 0.0, -0.4905010756171413, -0.098101...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.49..."
3,7405,NO 15/9-F-1 C,2014-04-25,0.268405,0.705031,0.642083,0.161491,1.145565,-0.379061,2.364038,...,0.024781,-0.133505,-0.264997,-0.368282,-0.001433,-0.0027,-0.005271,"[-0.4905010756171413, -0.09810136648612545, 0....","[0.0, 0.0, -0.4905010756171413, -0.09810136648...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.4905010..."
4,7405,NO 15/9-F-1 C,2014-04-26,0.268405,0.625355,0.643889,0.117219,1.145565,-0.340115,2.088747,...,0.057726,-0.077878,-0.210807,-0.330137,-0.084172,-0.0027,-0.005271,"[-0.09810136648612545, 0.18307478352672518, -0...","[0.0, -0.4905010756171413, -0.0981013664861254...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.490501075617..."


In [304]:
well_info = dict(set([(j, i) for i, j in data[['WELL_BORE_CODE', 'well_name']].values.tolist()])) # creating a dictionary of well identiification with {code: name} structure
well_info

{7405: 'NO 15/9-F-1 C',
 7078: 'NO 15/9-F-11 H',
 5599: 'NO 15/9-F-12 H',
 7289: 'NO 15/9-F-15 D',
 5351: 'NO 15/9-F-14 H',
 5769: 'NO 15/9-F-5 AH'}

In [305]:
data.columns

Index(['well_name', 'WELL_BORE_CODE', 'date', 'prod_hrs', 'bhp', 'bht',
       'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht',
       'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'oil_rate',
       'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long',
       'Rolling_short', 'Rolling_medium', 'Rolling_long', 'Lag_short',
       'Lag_medium', 'Lag_long'],
      dtype='object')

In [306]:
identification_column = ['well_name', 'WELL_BORE_CODE', 'date']
single_feature_columns = ['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']
multiple_feature_columns = ['Lag_short', 'Lag_medium', 'Lag_long']
feature_columns = single_feature_columns + multiple_feature_columns
target_column = ['oil_rate']

In [307]:
data = data[identification_column + feature_columns]
data.shape

(7987, 26)

In [308]:
print(data[['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']].mean().round(4))
print(data[['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']].std().round(4))

prod_hrs                -0.0
bhp                      0.0
bht                     -0.0
dp_tubing                0.0
AVG_ANNULUS_PRESS        0.0
AVG_CHOKE_SIZE_P        -0.0
whp                     -0.0
wht                     -0.0
choke_size_percentage    0.0
oil_vol                  0.0
gas_vol                  0.0
water_vol                0.0
linear_regressor        -0.0
EMA_short                0.0
EMA_medium              -0.0
EMA_long                 0.0
Rolling_short           -0.0
Rolling_medium           0.0
Rolling_long             0.0
oil_rate                -0.0
dtype: float64
prod_hrs                 1.0
bhp                      1.0
bht                      1.0
dp_tubing                1.0
AVG_ANNULUS_PRESS        1.0
AVG_CHOKE_SIZE_P         1.0
whp                      1.0
wht                      1.0
choke_size_percentage    1.0
oil_vol                  1.0
gas_vol                  1.0
water_vol                1.0
linear_regressor         1.0
EMA_short                1.0

In [309]:
data

,well_name,WELL_BORE_CODE,date,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,...,EMA_short,EMA_medium,EMA_long,Rolling_short,Rolling_medium,Rolling_long,oil_rate,Lag_short,Lag_medium,Lag_long
0,7405,NO 15/9-F-1 C,2014-04-22,0.268405,1.029309,0.605973,0.396165,-2.015211,-0.533280,3.281836,...,-0.495877,-0.498713,-0.502834,-0.001433,-0.002700,-0.005271,-0.490501,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,7405,NO 15/9-F-1 C,2014-04-23,0.268405,0.845793,0.634777,0.239816,0.088574,-0.419656,2.849981,...,-0.363747,-0.426301,-0.464667,-0.001433,-0.002700,-0.005271,-0.098101,"[0.0, 0.0, -0.4905010756171413]","[0.0, 0.0, 0.0, 0.0, -0.4905010756171413]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7405,NO 15/9-F-1 C,2014-04-24,0.268405,0.775023,0.639809,0.199889,-0.687843,-0.402874,2.607708,...,-0.180982,-0.315168,-0.402787,-0.001433,-0.002700,-0.005271,0.183075,"[0.0, -0.4905010756171413, -0.09810136648612545]","[0.0, 0.0, 0.0, -0.4905010756171413, -0.098101...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.49..."
3,7405,NO 15/9-F-1 C,2014-04-25,0.268405,0.705031,0.642083,0.161491,1.145565,-0.379061,2.364038,...,-0.133505,-0.264997,-0.368282,-0.001433,-0.002700,-0.005271,-0.037781,"[-0.4905010756171413, -0.09810136648612545, 0....","[0.0, 0.0, -0.4905010756171413, -0.09810136648...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.4905010..."
4,7405,NO 15/9-F-1 C,2014-04-26,0.268405,0.625355,0.643889,0.117219,1.145565,-0.340115,2.088747,...,-0.077878,-0.210807,-0.330137,-0.084172,-0.002700,-0.005271,0.033425,"[-0.09810136648612545, 0.18307478352672518, -0...","[0.0, -0.4905010756171413, -0.0981013664861254...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.490501075617..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7982,5769,NO 15/9-F-5 AH,2016-08-22,0.268405,-0.880933,-0.806036,-0.814176,1.149547,1.150153,-1.140330,...,-0.693212,-0.702888,-0.716277,-0.692951,-0.704178,-0.725482,-0.681306,"[-0.6834479858098645, -0.6925870358402809, -0....","[-0.6915821804196012, -0.6860958165169114, -0....","[-0.7117159623886966, -0.7111878631895072, -0...."
7983,5769,NO 15/9-F-5 AH,2016-08-23,0.268405,-0.880933,-0.806036,-0.814176,1.159077,1.150153,-1.136215,...,-0.689899,-0.699999,-0.713993,-0.690916,-0.700610,-0.723721,-0.676011,"[-0.6925870358402809, -0.6822891014560878, -0....","[-0.6860958165169114, -0.6834479858098645, -0....","[-0.7111878631895072, -0.7027822842690761, -0...."
7984,5769,NO 15/9-F-5 AH,2016-08-24,0.268405,-0.880933,-0.806036,-0.814176,1.141368,1.150153,-1.137527,...,-0.688108,-0.697863,-0.712046,-0.689665,-0.698020,-0.721490,-0.677250,"[-0.6822891014560878, -0.6813062501687076, -0....","[-0.6834479858098645, -0.6925870358402809, -0....","[-0.7027822842690761, -0.695718957479918, -0.6..."
7985,5769,NO 15/9-F-5 AH,2016-08-25,0.268405,-1.334973,-1.285316,-1.327392,1.140972,1.150153,-1.140776,...,-0.686771,-0.696038,-0.710244,-0.686483,-0.696104,-0.719541,-0.676825,"[-0.6813062501687076, -0.6760105887546141, -0....","[-0.6925870358402809, -0.6822891014560878, -0....","[-0.695718957479918, -0.689095713356751, -0.69..."


In [310]:
for column in multiple_feature_columns:
    try:
        data[column] = data[column].apply(ast.literal_eval)
    except Exception:
        pass

In [311]:
data.isna().any()

well_name                False
WELL_BORE_CODE           False
date                     False
prod_hrs                 False
bhp                      False
bht                      False
dp_tubing                False
AVG_ANNULUS_PRESS        False
AVG_CHOKE_SIZE_P         False
whp                      False
wht                      False
choke_size_percentage    False
oil_vol                  False
gas_vol                  False
water_vol                False
linear_regressor         False
EMA_short                False
EMA_medium               False
EMA_long                 False
Rolling_short            False
Rolling_medium           False
Rolling_long             False
oil_rate                 False
Lag_short                False
Lag_medium               False
Lag_long                 False
dtype: bool

In [312]:
# Get the numerical index of the column 'oil_rate'
column_name = 'oil_rate'
column_index = data[feature_columns].columns.get_loc(column_name)
column_index

19

In [313]:
batch_size = 96
days_window = 5
predictions_days = 14
training_gap = predictions_days + days_window

In [314]:
# sequences = []
# for well_name in data['well_name'].unique():
#     well_data = data[data['well_name'] == well_name].sort_values(by='date').reset_index()
#     for i in range(len(well_data) - training_gap + 1):
#         single_features = np.array(well_data.iloc[i:i + days_window][single_feature_columns])
#         multiple_features = np.array(well_data.iloc[i:i + days_window][multiple_feature_columns])
#         multiple_flattened = []
#         for k in multiple_features:
#            flattened_list = [item for sublist in k for item in sublist]
#            multiple_flattened.append(flattened_list)
#         multiple_flattened = np.array(multiple_flattened)
#         inputs = np.concatenate((single_features, multiple_flattened), axis=1)
#         targets = well_data.iloc[i + days_window:i + days_window + predictions_days][target_column].values
#         inputs_dates = list(well_data.iloc[i:i + days_window]['date'])
#         targets_dates = list(well_data.iloc[i + days_window:i + days_window + predictions_days]['date'])
#         inputs_index = list(well_data.iloc[i:i + days_window]['index'])
#         targets_index = list(well_data.iloc[i + days_window:i + days_window + predictions_days]['index'])
#         sequences.append((well_name, inputs_dates, targets_dates, inputs_index, targets_index, inputs, targets))
#         if len(well_data.iloc[i:i + days_window + predictions_days]['well_name'].unique()) > 1:
#           print('violation')

# dataset = pd.DataFrame(sequences, columns=['well_name', 'inputs_dates', 'targets_dates', 'inputs_index', 'targets_index', 'inputs', 'targets', ])
# dataset.to_pickle(path + "/data/Volve/all_data.pkl")
# dataset.to_csv(path + "/data/Volve/all_data.csv")
# dataset

,well_name,inputs_dates,targets_dates,inputs_index,targets_index,inputs,targets
0,7405,"[2014-04-22, 2014-04-23, 2014-04-24, 2014-04-2...","[2014-04-27, 2014-04-28, 2014-04-29, 2014-04-3...","[0, 1, 2, 3, 4]","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[[0.2684054414666464, 1.0293086129008744, 0.60...","[[0.0361971937521761], [0.0332999828677342], [..."
1,7405,"[2014-04-23, 2014-04-24, 2014-04-25, 2014-04-2...","[2014-04-28, 2014-04-29, 2014-04-30, 2014-05-0...","[1, 2, 3, 4, 5]","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[[0.2684054414666464, 0.8457926690107068, 0.63...","[[0.0332999828677342], [-0.0152191310577907], ..."
2,7405,"[2014-04-24, 2014-04-25, 2014-04-26, 2014-04-2...","[2014-04-29, 2014-04-30, 2014-05-01, 2014-05-0...","[2, 3, 4, 5, 6]","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, ...","[[0.2684054414666464, 0.7750228696616299, 0.63...","[[-0.0152191310577907], [-0.0547092156193974],..."
3,7405,"[2014-04-25, 2014-04-26, 2014-04-27, 2014-04-2...","[2014-04-30, 2014-05-01, 2014-05-02, 2014-05-0...","[3, 4, 5, 6, 7]","[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...","[[0.2684054414666464, 0.7050308265409722, 0.64...","[[-0.0547092156193974], [-0.0640389681384098],..."
4,7405,"[2014-04-26, 2014-04-27, 2014-04-28, 2014-04-2...","[2014-05-01, 2014-05-02, 2014-05-03, 2014-05-0...","[4, 5, 6, 7, 8]","[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20...","[[0.2684054414666464, 0.6253548436655879, 0.64...","[[-0.0640389681384098], [-0.1658887664709635],..."
...,...,...,...,...,...,...,...
7874,5769,"[2016-08-04, 2016-08-05, 2016-08-06, 2016-08-0...","[2016-08-09, 2016-08-10, 2016-08-11, 2016-08-1...","[7964, 7965, 7966, 7967, 7968]","[7969, 7970, 7971, 7972, 7973, 7974, 7975, 797...","[[0.2684054414666464, -0.475522612140657, -0.3...","[[-0.730276391937718], [-0.7220799091727883], ..."
7875,5769,"[2016-08-05, 2016-08-06, 2016-08-07, 2016-08-0...","[2016-08-10, 2016-08-11, 2016-08-12, 2016-08-1...","[7965, 7966, 7967, 7968, 7969]","[7970, 7971, 7972, 7973, 7974, 7975, 7976, 797...","[[0.2684054414666464, -0.475522612140657, -0.3...","[[-0.7220799091727883], [-0.7175543924241792],..."
7876,5769,"[2016-08-06, 2016-08-07, 2016-08-08, 2016-08-0...","[2016-08-11, 2016-08-12, 2016-08-13, 2016-08-1...","[7966, 7967, 7968, 7969, 7970]","[7971, 7972, 7973, 7974, 7975, 7976, 7977, 797...","[[0.2684054414666464, -0.475522612140657, -0.3...","[[-0.7175543924241792], [-0.7117159623886966],..."
7877,5769,"[2016-08-07, 2016-08-08, 2016-08-09, 2016-08-1...","[2016-08-12, 2016-08-13, 2016-08-14, 2016-08-1...","[7967, 7968, 7969, 7970, 7971]","[7972, 7973, 7974, 7975, 7976, 7977, 7978, 797...","[[0.2684054414666464, -0.475522612140657, -0.3...","[[-0.7117159623886966], [-0.7111878631895072],..."


In [315]:
train_size = 0.6
val_size = 0.2
test_size = 0.2

In [316]:
test_size_param1 = test_size
test_size_param2 = val_size / (1 - test_size_param1)

In [317]:
# For shuffled train/val

# dataset = pd.read_pickle(path + "/data/Volve/all_data.pkl")
train_wells = []
test_wells = []
for i in well_info.keys():
    well = dataset[dataset['well_name'] == i]
    train_set_length = int((train_size + val_size) * len(well))
    train_wells.append(well[:train_set_length + 1])
    test_set_length = int(test_size * len(well))
    test_wells.append(well[-test_set_length:])

# Flatten the lists of DataFrames
temp_data = pd.concat(train_wells).reset_index(drop=True)
data_test = pd.concat(test_wells).reset_index(drop=True)


data_train, data_val = train_test_split(temp_data, test_size=test_size_param2, stratify=temp_data['well_name'], shuffle=True)

data_train.reset_index(drop=True, inplace=True)
data_val.reset_index(drop=True, inplace=True)
data_test.reset_index(drop=True, inplace=True)

data_train.to_pickle(path + "/data/Volve/data_train.pkl")
data_val.to_pickle(path + "/data/Volve/data_val.pkl")
data_test.to_pickle(path + "/data/Volve/data_test.pkl")

data_train.to_csv(path + "/data/Volve/data_train.csv")
data_val.to_csv(path + "/data/Volve/data_val.csv")
data_test.to_csv(path + "/data/Volve/data_test.csv")


In [124]:
# For in order train/val

# dataset = pd.read_pickle(path + "/data/Volve/all_data.pkl")

# # Initialize lists to store DataFrames for each well
# train_wells = []
# val_wells = []
# test_wells = []

# # Iterate over each well and split the data
# for well_name in well_info.keys():
#     well = dataset[dataset['well_name'] == well_name]
#     train_set_length = int(train_size * len(well))
#     val_set_length = int(val_size * len(well))
#     test_set_length = int(test_size * len(well))
    
#     train_wells.append(well[:train_set_length])
#     val_wells.append(well[train_set_length:train_set_length + val_set_length])
#     test_wells.append(well[-test_set_length:])

# # Flatten the lists of DataFrames
# data_train = pd.concat(train_wells).reset_index(drop=True)
# data_val = pd.concat(val_wells).reset_index(drop=True)
# data_test = pd.concat(test_wells).reset_index(drop=True)

In [318]:
all_data = pd.read_pickle(path + "/data/Volve/all_data.pkl")
data_train = pd.read_pickle(path + "/data/Volve/data_train.pkl")
data_val = pd.read_pickle(path + "/data/Volve/data_val.pkl")
data_test = pd.read_pickle(path + "/data/Volve/data_test.pkl")

In [258]:
# all_data = pd.read_csv(path + "/data/Volve/all_data.csv")
# data_train = pd.read_csv(path + "/data/Volve/data_train.csv")
# data_val = pd.read_csv(path + "/data/Volve/data_val.csv")
# data_test = pd.read_csv(path + "/data/Volve/data_test.csv")

In [221]:
# for df in [all_data, data_train, data_val, data_test]:
#     # Ensure each entry in 'inputs' and 'targets' is a NumPy array
#     df['inputs'] = df['inputs'].apply(lambda x: np.array(x, dtype=np.float32))
#     df['targets'] = df['targets'].apply(lambda x: np.array(x, dtype=np.float32))

In [284]:
data_train.shape, data_val.shape, data_test.shape

((4730, 7), (1577, 7), (1573, 7))

In [319]:
check = True
for i, j in well_info.items():
    if len(all_data[all_data['well_name'] == i]) < 10:
        print("The input/output gap is too high for well", i)
        check = False
if check:
    print("The input/output gap is fine")

The input/output gap is fine


In [320]:
check=True
for i in [data_train, data_val, data_test]:
    for j in well_info.keys():
        if j not in set(list(i['well_name'])):
            print("Some datasets do not have specific well")
            check = False
            break
if check:
    print("All datasets have all wells")

All datasets have all wells


In [270]:
for df in [data_train, data_val, data_test]:
    for column in df.columns:
        try:
            data[column] = data[column].apply(ast.literal_eval)
        except Exception:
            pass

In [322]:
data_train['inputs'][0].shape, data_train['targets'][0].shape

((5, 38), (14, 1))

In [323]:
if torch.backends.mps.is_available():
    device = 'mps'
elif torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu' 
device

'mps'

In [324]:
eval_interval = 500
test_eval_interval = 500
test_iterations = 7500
warmup_steps = 6000 # hyper-parameters for hyperprameter finetuning

In [325]:
class LSTM(nn.Module):
    def __init__(self, input_length, hidden_length, output_length, num_layers, dropout=0):
        super().__init__()
        self.input_length = input_length
        self.hidden_length = hidden_length
        self.output_length = output_length
        self.init_hidden = nn.Parameter(torch.zeros(num_layers, 1, hidden_length))
        self.init_cell = nn.Parameter(torch.zeros(num_layers, 1, hidden_length))
        self.num_layers = num_layers
        self.lstm_cell = nn.LSTM(self.input_length, self.hidden_length, self.num_layers, batch_first=True)
        self.output_layer = nn.Linear(self.hidden_length, self.output_length)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, targets=None):
        B, T, C = x.shape
        init_hidden = self.init_hidden.repeat(1, B, 1)
        init_cell = self.init_cell.repeat(1, B, 1)
        cell_output, _ = self.lstm_cell(x, (init_hidden, init_cell))
        cell_dropout = self.dropout(cell_output)
        logits = self.output_layer(cell_dropout[:, -1, :])
        if targets is None:
            loss = None
        else:
            B, T = logits.shape
            logits = logits.view(B*T)
            targets = targets.view(B*T)
            loss = F.mse_loss(logits, targets)
        return logits, loss

    def get_hyperparameters(self):
        return {'input_length': self.input_length, 'hidden_length': self.hidden_length, 'output_length': self.output_length, 'num_layers': self.num_layers, 'dropout': self.dropout}

In [326]:
class TimeSeriesDataset(Dataset):
    def __init__(self, sequences_df):
        self.sequences_df = sequences_df

    def __len__(self):
        return len(self.sequences_df)

    def __getitem__(self, idx):
        inputs = self.sequences_df.iloc[idx]['inputs']
        targets = self.sequences_df.iloc[idx]['targets']
        torch.tensor(targets, dtype=torch.float32).squeeze()
        # return torch.tensor(np.array(inputs, dtype=np.float32), dtype=torch.float32), torch.tensor(np.array(targets, dtype=np.float32), dtype=torch.float32).squeeze()
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32).squeeze()


train_dataset = TimeSeriesDataset(data_train)
val_dataset = TimeSeriesDataset(data_val)
test_dataset = TimeSeriesDataset(data_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [327]:
class VisualizationTimeSeriesDataset(Dataset):
    def __init__(self, sequences_df):
        self.sequences_df = sequences_df

    def __len__(self):
        return len(self.sequences_df)

    def __getitem__(self, idx):
        inputs = self.sequences_df.iloc[idx]['inputs']
        targets = self.sequences_df.iloc[idx]['targets']
        well_name = self.sequences_df.iloc[idx]['well_name']
        inputs_dates = self.sequences_df.iloc[idx]['inputs_dates']
        targets_dates = self.sequences_df.iloc[idx]['targets_dates']

        return well_name, inputs_dates, targets_dates, torch.tensor(inputs, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32).squeeze()

visualization_test_dataset = VisualizationTimeSeriesDataset(data_test)

visualization_test_loader = DataLoader(visualization_test_dataset, batch_size=int(len(test_dataset) / 50), shuffle=True)

In [328]:
input_length = data_train['inputs'][0].shape[1]
output_length = predictions_days

In [337]:
@torch.no_grad()
def estimate_loss(model): # Function to estimate train and val loss while training
    model.eval()  # Set model to evaluation mode
    out = {}
    loaders = {'train': train_loader, 'val': val_loader}
    for mode in ['train', 'val']:
        losses = []
        # Iterate over the DataLoader for a fixed number of batches
        for X_batch, Y_batch in loaders[mode]:
            X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
            logits, loss = model(X_batch, Y_batch)  # Run inputs and expected outputs through model to get loss
            losses.append(loss.item())  # Add value to losses list
        out[mode] = torch.tensor(losses).mean().item()  # Get mean of losses tensor and save it
    model.train()  # Put model back in train mode
    return out

@torch.no_grad()
def save_params(model, optimizer, scheduler, name=""): # function to save parameters, optimizer and scheduler states
    config_path = path + r'/Models/Global/LSTM/Model/State/'
    model_path = path + r'/Models/Global/LSTM/Model/Complete/'
    if name != "":
        name = f'{name}_'
    torch.save(model.state_dict(), config_path + f'{name}state_params.pth') # to save parameters
    torch.save(optimizer.state_dict(), config_path + f'{name}state_optimizer.pth') # to save optimizer state
    torch.save(scheduler.state_dict(), config_path + f'{name}state_scheduler.pth') # to save scheduler states
    torch.save(model, model_path + f'{name}complete_params.pth')
    torch.save(optimizer, model_path + f'{name}complete_optimizer.pth') 
    torch.save(scheduler, model_path + f'{name}complete_scheduler.pth')

def get_next_log_file(log_dir, prefix="training_run_"):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
        return os.path.join(log_dir, f"{prefix}1.txt")
    existing_files = os.listdir(log_dir)
    log_numbers = []
    for filename in existing_files:
        match = re.match(rf"{prefix}(\d+).txt", filename)
        if match:
            log_numbers.append(int(match.group(1)))
    if log_numbers:
        next_number = max(log_numbers) + 1
    else:
        next_number = 1
    return os.path.join(log_dir, f"{prefix}{next_number}.txt")

def log_to_file(log_file, message):
    with open(log_file, 'a') as f:
        f.write(message + '\n')

def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2}

@torch.no_grad()
def evaluate(mode, model):
    loader_dict = {'train': train_loader, 'val': val_loader, 'test': test_loader}
    loader = loader_dict[mode]
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_targets = []
    test_losses = []
    for X_batch, Y_batch in loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        logits, loss = model(X_batch, Y_batch)  # Run inputs and expected outputs through model to get loss
        test_losses.append(loss.item())  # Collect test loss
        B, T = Y_batch.shape
        Y_batch = Y_batch.view(B*T)
        # Convert predictions and targets to NumPy arrays
        all_preds.append(logits.cpu().numpy())  # .cpu() moves tensor to CPU, .numpy() converts to NumPy array
        all_targets.append(Y_batch.cpu().numpy())
    # Concatenate all predictions and targets
    all_preds = np.concatenate(all_preds, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)
    # Calculate metrics
    metrics = calculate_metrics(all_targets, all_preds)
    metrics['Loss'] = np.mean(test_losses)
    print(f"Metrics on {mode} set:\n")
    print(f"Loss is {metrics['Loss']:.4f}")
    print(f"MAE is {metrics['MAE']:.4f}")
    print(f"RMSE is {metrics['RMSE']:.4f}")
    print(f"R² is {metrics['R2']:.4f}")
    print("\n")
    return metrics


# Define an objective function for Optuna
def objective(trial):
    vector_length = trial.suggest_int('vector_length', 16, 2048, step=16)
    num_layers = trial.suggest_int('num_layers', 1, 5) # maybe increase it to 6
    dropout = trial.suggest_float('dropout', 0.1, 0.6)
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    gamma = trial.suggest_float('gamma', 0.95, 0.99)
    model = LSTM(input_length=input_length, hidden_length=vector_length, output_length=output_length, num_layers=num_layers, dropout=dropout).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma) # setting optimizer scheduler
    train_loader_iter = iter(train_loader)
    # Training loop
    for i in range(test_iterations):
        model.train()
        if (i % test_eval_interval == 0 or i == test_iterations - 1):
            losses = estimate_loss(model)
            trial.report(losses['val'], i)
            if i != 0:
                scheduler.step()
            # if i >= warmup_steps and trial.should_prune():
            #     raise optuna.TrialPruned()
        try:
            X_batch, Y_batch = next(train_loader_iter)
        except StopIteration:
            train_loader_iter = iter(train_loader)
            X_batch, Y_batch = next(train_loader_iter)
            
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        logits, loss = model(X_batch, Y_batch)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
    val_metrics = evaluate('val', model)
    return val_metrics['Loss']

def load_model(name=""): 
    config_path = path + r'/Models/Global/LSTM/Model/Complete/'
    if name != "":
        name = f'{name}_'
    model = torch.load(config_path + f'{name}complete_params.pth')
    optimizer = torch.load(config_path + f'{name}complete_optimizer.pth')
    scheduler = torch.load(config_path + f'{name}complete_scheduler.pth')
    return model, optimizer, scheduler

def get_oil_history(date, well_name):
    well = data[data["well_name"] == int(well_name)].reset_index(drop=True)
    well = well[pd.to_datetime(well['date']) < date]
    return well['oil_rate']

def rename(config_path, model_path, old_name, new_name):
    os.rename(config_path + f'{old_name}state_params.pth', config_path + f'{new_name}state_params.pth')
    os.rename(config_path + f'{old_name}state_optimizer.pth', config_path + f'{new_name}state_optimizer.pth')
    os.rename(config_path + f'{old_name}state_scheduler.pth', config_path + f'{new_name}state_scheduler.pth')
    os.rename(model_path + f'{old_name}complete_params.pth', model_path + f'{new_name}complete_params.pth')
    os.rename(model_path + f'{old_name}complete_optimizer.pth', model_path + f'{new_name}complete_optimizer.pth')
    os.rename(model_path + f'{old_name}complete_scheduler.pth', model_path + f'{new_name}complete_scheduler.pth')
    
def label_model_files(old_name="", new_name=""):
    if old_name != "":
        old_name = f'{old_name}_'
    if new_name != "":
        new_name = f'{new_name}_'
    config_path = path + r'/Models/Global/LSTM/Model/State/'
    model_path = path + r'/Models/Global/LSTM/Model/Complete/'
    if os.path.exists(config_path + f'{new_name}state_params.pth'):
        try:
            user_choice = int(input("File with new name already exists. Enter 1 if you want to overwrite the files. Otherwise press 2: "))
        except ValueError:
            print("Invalid input. Please enter 1 or 2.")
            return
        if user_choice == 1:
            try:
                rename(config_path, model_path, old_name, new_name)
                print("Files have been renamed successfully.")
            except FileNotFoundError:
                print("One or more files not found.")
        elif user_choice == 2:
            print("Operation cancelled by the user.")
        else:
            print("Invalid choice. Operation cancelled.")
    else:
        try:
            rename(config_path, model_path, old_name, new_name)
            print("Files have been renamed successfully.")
        except FileNotFoundError:
            print("One or more files not found.")

def predictions_with_history_graphs(model, visualization_loader, path):
    pdf = PdfPages(path + "/predictions_with_history.pdf")
    with torch.no_grad():
        model.eval()
        for well_name, inputs_dates, targets_dates, X_batch, Y_batch in visualization_loader:
            X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
            choice = random.randint(0, len(X_batch) - 1)
            X_batch = X_batch[choice].unsqueeze(0)
            Y_batch = Y_batch[choice]
            well_name = well_name[choice]
            inputs_dates = [i[choice] for i in inputs_dates]
            targets_dates = [i[choice] for i in targets_dates]

            history = get_oil_history(pd.to_datetime(targets_dates[0]), well_name)
            logits, loss = model(X_batch, Y_batch)
            label = Y_batch.cpu().numpy()
            prediction = logits.cpu().numpy()
            rmse = np.sqrt(mean_squared_error(label, prediction))
            # Plotting
            forecast_index = range(len(history) + 1, len(history) + predictions_days + 1)
            plt.figure(figsize=(12, 6))
            plt.plot(range(1, len(history) + 1), history, label='Input Values')
            plt.plot(forecast_index, prediction, label='Forecasted Values')
            plt.plot(forecast_index, label, label='Actual Values')
            # Determine the interval for ticks to have approximately 10 ticks
            tick_interval = max(10, round((len(history) + predictions_days + 1) / 10 / 10) * 10)
            # Generate tick positions
            tick_positions = np.arange(0, len(history) + predictions_days + 1, tick_interval)
            tick_positions[0] = 1  # Start tick at 1 instead of 0

            # Generate tick labels
            tick_labels = tick_positions
            plt.xticks(ticks=tick_positions, labels=tick_labels)
            plt.axvline(x=len(history), color='r', linestyle='--', label='Forecast Start')
            plt.xlabel('Day')
            plt.ylabel('Value')
            plt.title(f'Well: {well_name}, Loss: {loss:.4f}, RMSE = {rmse:.4f}')
            plt.legend()
            plt.grid(True)
            # Save the current figure to the PDF
            pdf.savefig()
            plt.close()
    pdf.close()

def predictions_graphs(model, visualization_loader, path):
    pdf = PdfPages(path + "/predictions.pdf")
    with torch.no_grad():
        model.eval()
        for well_name, inputs_dates, targets_dates, X_batch, Y_batch in visualization_loader:
            X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
            choice = random.randint(0, len(X_batch) - 1)
            X_batch = X_batch[choice].unsqueeze(0)
            Y_batch = Y_batch[choice]
            well_name = well_name[choice]
            logits, loss = model(X_batch, Y_batch)
            inputs = X_batch[0][:, column_index].cpu().numpy()
            label = Y_batch.cpu().numpy()
            prediction = logits.cpu().numpy()
            rmse = np.sqrt(mean_squared_error(label, prediction))
            # Plotting
            forecast_index = range(len(inputs) + 1, len(inputs) + predictions_days + 1)
            plt.figure(figsize=(12, 6))
            plt.plot(range(1, len(inputs) + 1), inputs, marker='o', label='Input Values')
            plt.plot(forecast_index, prediction, marker='o', label='Forecasted Values')
            plt.plot(forecast_index, label, marker='o', label='Actual Values')
            plt.xticks(ticks=np.arange(1, len(inputs) + predictions_days + 1), labels=np.arange(1, len(inputs) + predictions_days + 1))
            plt.axvline(x=len(inputs), color='r', linestyle='--', label='Forecast Start')
            plt.xlabel('Day')
            plt.ylabel('Value')
            plt.title(f'Well: {well_name}, Loss: {loss:.4f}, RMSE: {rmse:.4f}')
            plt.legend()
            plt.grid(True)
            # Save the current figure to the PDF
            pdf.savefig()
            plt.close()
    pdf.close()

def all_predictions_graphs(model, data, data_test, path):
    well_dataloaders = {}
    pdf = PdfPages(path + "/all_predictions.pdf")
    with torch.no_grad():
        for well_name in well_info.keys():
            well = data[data['well_name'] == well_name].reset_index(drop=True)
            fig, ax = plt.subplots(figsize=(15, 6))
            ax.plot(well.index, well['oil_rate'], label='Well Production')
            ax.axvline(x=int((1 - test_size) * len(well) - days_window - 2), color='r', linestyle='--',)
            ax.legend()
            well_dataset = VisualizationTimeSeriesDataset(data_test[data_test['well_name'] == well_name].reset_index(drop=True))
            well_dataloaders[well_name] = DataLoader(well_dataset, batch_size=1, shuffle=False)
            total_loss = 0
            total_rmse = 0
            for _, inputs_dates, targets_dates, X_batch, Y_batch in well_dataloaders[well_name]:
                inputs_dates = np.array(inputs_dates).T[0]
                targets_dates = np.array(targets_dates).T[0]
                history = get_oil_history(pd.to_datetime(targets_dates[0]), well_name)
                X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
                logits, loss = model(X_batch, Y_batch)
                label = Y_batch.cpu().numpy().reshape(-1)
                prediction = logits.cpu().numpy()
                rmse = np.sqrt(mean_squared_error(label, prediction))
                total_loss += loss
                total_rmse += rmse
                forecast_index = range(len(history) + 1, len(history) + predictions_days + 1)
                ax.plot(forecast_index, prediction, label='Forecasted Values', color='y')
                if i == 0:
                    ax.legend()
            avg_loss = total_loss / len(well_dataloaders[well_name]) 
            avg_rmse = total_rmse / len(well_dataloaders[well_name]) 
            ax.set_xlabel('Day')
            ax.set_ylabel('Value')
            ax.set_title(f'Well: {well_name}, Loss: {avg_loss:.4f}, RMSE = {avg_rmse:.4f}')
            pdf.savefig(fig)
            plt.close()
    pdf.close()

def all_predictions_with_gaps_graphs(model, data, data_test, path):
    well_plots = {}
    well_dataloaders = {}   
    pdf = PdfPages(path + "/all_predictions_with_gaps.pdf")
    with torch.no_grad():
        for well_name in well_info.keys():
            well = data[data['well_name'] == well_name].reset_index(drop=True)
            fig, ax = plt.subplots(figsize=(15, 6))
            ax.plot(well.index, well['oil_rate'], label='Well Production')
            ax.axvline(x=int((1 - test_size) * len(well) - days_window - 2), color='r', linestyle='--',)
            ax.legend()
            well_dataset = VisualizationTimeSeriesDataset(data_test[data_test['well_name'] == well_name].reset_index(drop=True))
            well_dataloaders[well_name] = DataLoader(well_dataset, batch_size=1, shuffle=False)
            total_loss = 0
            total_rmse = 0
            for i, (_, inputs_dates, targets_dates, X_batch, Y_batch) in enumerate(well_dataloaders[well_name]):
                if i % predictions_days == 0 or i == len(well_dataloaders[well_name]) - 1:
                    inputs_dates = np.array(inputs_dates).T[0]
                    targets_dates = np.array(targets_dates).T[0]
                    history = get_oil_history(pd.to_datetime(targets_dates[0]), well_name)
                    X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
                    logits, loss = model(X_batch, Y_batch)
                    label = Y_batch.cpu().numpy().reshape(-1)
                    prediction = logits.cpu().numpy()
                    rmse = np.sqrt(mean_squared_error(label, prediction))
                    total_loss += loss
                    total_rmse += rmse
                    forecast_index = range(len(history) + 1, len(history) + predictions_days + 1)
                    ax.plot(forecast_index, prediction, label='Forecasted Values', color='y')
                    if i == 0:
                        ax.legend()
            avg_loss = total_loss / len(well_dataloaders[well_name]) 
            avg_rmse = total_rmse / len(well_dataloaders[well_name]) 
            ax.set_xlabel('Day')
            ax.set_ylabel('Value')
            ax.set_title(f'Well: {well_name}, Loss: {avg_loss:.4f}, RMSE = {avg_rmse:.4f}')
            pdf.savefig(fig)
            plt.close()
    pdf.close()

In [338]:
# Create a study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

# Get the best hyperparameters
best_params = study.best_params
print('Best hyperparameters:', best_params)

# Train the final model with the best hyperparameters
vector_length = best_params['vector_length']
num_layers = best_params['num_layers']
dropout = best_params['dropout']
lr = best_params['lr']
gamma = best_params['gamma']

[I 2024-07-15 15:40:07,644] A new study created in memory with name: no-name-5f93a5fc-7fd6-47b8-a694-b046706fe8fa
[I 2024-07-15 15:42:04,866] Trial 0 finished with value: 0.03663919230594354 and parameters: {'vector_length': 640, 'num_layers': 2, 'dropout': 0.19641344503635347, 'lr': 0.002949014868913066, 'gamma': 0.9664126117454551}. Best is trial 0 with value: 0.03663919230594354.


Metrics on val set:

Loss is 0.0366
MAE is 0.0944
RMSE is 0.1928
R² is 0.9661




[I 2024-07-15 15:44:26,417] Trial 1 finished with value: 0.039217724190915335 and parameters: {'vector_length': 272, 'num_layers': 4, 'dropout': 0.5567765324379755, 'lr': 0.0003826257965089424, 'gamma': 0.9725948077074607}. Best is trial 0 with value: 0.03663919230594354.


Metrics on val set:

Loss is 0.0392
MAE is 0.0970
RMSE is 0.1950
R² is 0.9653




[I 2024-07-15 15:47:22,617] Trial 2 finished with value: 0.037502232829437536 and parameters: {'vector_length': 704, 'num_layers': 4, 'dropout': 0.23433615429117358, 'lr': 0.0005493286657334334, 'gamma': 0.9845257842874103}. Best is trial 0 with value: 0.03663919230594354.


Metrics on val set:

Loss is 0.0375
MAE is 0.0925
RMSE is 0.1912
R² is 0.9666




In [233]:
vector_length = 1024
num_layers = 3
dropout = 0.19057012152026273
lr = 0.0001911914637460164
gamma = 0.9806898435934219 # 2nd best results on val set full training

In [330]:
vector_length = 336
num_layers = 3
dropout = 0.12831089625320793
lr = 0.0003724263259631224
gamma = 0.978770138138513 # best results on val set full training

In [331]:
data_train

,well_name,inputs_dates,targets_dates,inputs_index,targets_index,inputs,targets
0,5599,"[2012-02-07, 2012-02-08, 2012-02-09, 2012-02-1...","[2012-02-12, 2012-02-13, 2012-02-14, 2012-02-1...","[2883, 2884, 2885, 2886, 2887]","[2888, 2889, 2890, 2891, 2892, 2893, 2894, 289...","[[0.2684054414666464, -1.7396742091238298, -1....","[[-0.251118109451251], [-0.2404607742231651], ..."
1,5351,"[2014-11-07, 2014-11-08, 2014-11-09, 2014-11-1...","[2014-11-12, 2014-11-13, 2014-11-14, 2014-11-1...","[6489, 6490, 6491, 6492, 6493]","[6494, 6495, 6496, 6497, 6498, 6499, 6500, 650...","[[0.2684054414666464, 0.7586064622943015, 0.45...","[[-0.6863451966943063], [-0.6900785646441312],..."
2,7078,"[2014-02-21, 2014-02-22, 2014-02-23, 2014-02-2...","[2014-02-26, 2014-02-27, 2014-02-28, 2014-03-0...","[627, 628, 629, 630, 631]","[632, 633, 634, 635, 636, 637, 638, 639, 640, ...","[[0.2684054414666464, 0.5594461049563565, 0.60...","[[-0.0561028107283693], [-0.0663494021348634],..."
3,7078,"[2015-02-24, 2015-02-25, 2015-02-26, 2015-02-2...","[2015-03-01, 2015-03-02, 2015-03-03, 2015-03-0...","[989, 990, 991, 992, 993]","[994, 995, 996, 997, 998, 999, 1000, 1001, 100...","[[0.2684054414666464, 0.2748719780772813, 0.60...","[[-0.1789445522287011], [-0.1824285400011311],..."
4,5599,"[2009-07-08, 2009-07-09, 2009-07-10, 2009-07-1...","[2009-07-13, 2009-07-14, 2009-07-15, 2009-07-1...","[2045, 2046, 2047, 2048, 2049]","[2050, 2051, 2052, 2053, 2054, 2055, 2056, 205...","[[0.2684054414666464, 0.4820166611412672, 0.60...","[[1.6604616321034416], [1.6238980972706754], [..."
...,...,...,...,...,...,...,...
4725,7078,"[2015-04-15, 2015-04-16, 2015-04-17, 2015-04-1...","[2015-04-20, 2015-04-21, 2015-04-22, 2015-04-2...","[1039, 1040, 1041, 1042, 1043]","[1044, 1045, 1046, 1047, 1048, 1049, 1050, 105...","[[0.2684054414666464, 0.373529723656564, 0.588...","[[0.3777893590945128], [0.3903097109419611], [..."
4726,7289,"[2014-01-19, 2014-01-20, 2014-01-21, 2014-01-2...","[2014-01-24, 2014-01-25, 2014-01-26, 2014-01-2...","[7103, 7104, 7105, 7106, 7107]","[7108, 7109, 7110, 7111, 7112, 7113, 7114, 711...","[[0.2684054414666464, 0.2032044106079711, 0.60...","[[-0.6536103810556642], [-0.6281222599836762],..."
4727,7289,"[2014-11-11, 2014-11-12, 2014-11-13, 2014-11-1...","[2014-11-16, 2014-11-17, 2014-11-18, 2014-11-1...","[7382, 7383, 7384, 7385, 7386]","[7387, 7388, 7389, 7390, 7391, 7392, 7393, 739...","[[0.2684054414666464, 0.1758870184083419, 0.61...","[[-0.8149300169858227], [-0.8167270212052866],..."
4728,5351,"[2013-01-01, 2013-01-02, 2013-01-03, 2013-01-0...","[2013-01-06, 2013-01-07, 2013-01-08, 2013-01-0...","[5840, 5841, 5842, 5843, 5844]","[5845, 5846, 5847, 5848, 5849, 5850, 5851, 585...","[[0.2684054414666464, 0.5794212353501313, 0.49...","[[-0.2315784390812436], [-0.2295613935287841],..."


In [332]:
# Example usage
context_size = days_window - 1
X_batch, Y_batch = next(iter(train_loader))
X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
print(f'The shape of input is: {X_batch.shape} and the shape of the output is {Y_batch.shape}')
model = LSTM(input_length=input_length, hidden_length=vector_length, output_length=output_length, num_layers=num_layers, dropout=dropout).to(device)
output, loss = model(X_batch, Y_batch)
print(f'The output shape of the LSTM will be {tuple(output.shape)} and a an example initial loss is {loss}')

The shape of input is: torch.Size([96, 5, 38]) and the shape of the output is torch.Size([96, 14])
The output shape of the LSTM will be (1344,) and a an example initial loss is 0.9283939003944397


In [333]:
model = LSTM(input_length=input_length, hidden_length=vector_length, output_length=output_length, num_layers=num_layers, dropout=dropout).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma) # setting optimizer scheduler

In [334]:
iterations = 30000

In [335]:
train_loader_iter = iter(train_loader)
log_file = get_next_log_file(path + "/Models/Global/LSTM/Logs")

log_message = f"Hyperparameters: Vector Length = {vector_length}, Number of Layers = {num_layers}, Dropout = {dropout}, Learning Rate = {lr}, Scheduler Gamma = {gamma}\n"

log_to_file(log_file, log_message)
model.train()
try:
    for i in range(iterations):
        model.train()  # Set model to training mode
        if i % eval_interval == 0 or i == iterations - 1:
            # Evaluate and save model parameters periodically
            save_params(model, optimizer, scheduler, 'current')  # Saving parameters
            model.eval()
            losses = estimate_loss(model)  # Estimate train and val loss
            model.train()
            log_message = f"step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
            print(log_message)
            log_to_file(log_file, log_message)  # Log to file
            if i != 0:
                scheduler.step()  # Perform scheduler step
        try:
            # Get the next batch
            X_batch, Y_batch = next(train_loader_iter)
        except StopIteration:
            # Reinitialize the iterator if the DataLoader is exhausted
            train_loader_iter = iter(train_loader)
            X_batch, Y_batch = next(train_loader_iter)

        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        logits, loss = model(X_batch, Y_batch)  # Run inputs and expected outputs through model
        optimizer.zero_grad(set_to_none=True)  # Set gradients to 0 (to ensure gradients don't explode)
        loss.backward()  # Perform backpropagation
        optimizer.step()  # Perform parameter adjustment

except KeyboardInterrupt:
    print("Training ended manually by user.\n")
model.eval()
print("Training is complete")


step 0: train loss 1.1191, val loss 1.1372
step 500: train loss 0.0672, val loss 0.0662
step 1000: train loss 0.0620, val loss 0.0615
step 1500: train loss 0.0561, val loss 0.0570
step 2000: train loss 0.0512, val loss 0.0524
step 2500: train loss 0.0464, val loss 0.0499
step 3000: train loss 0.0410, val loss 0.0488
step 3500: train loss 0.0381, val loss 0.0467
step 4000: train loss 0.0303, val loss 0.0404
step 4500: train loss 0.0277, val loss 0.0406
step 5000: train loss 0.0244, val loss 0.0369
step 5500: train loss 0.0218, val loss 0.0368
step 6000: train loss 0.0200, val loss 0.0358
step 6500: train loss 0.0175, val loss 0.0362
Training ended manually by user.

Training is complete


In [213]:
model.eval()
train_metrics = evaluate('train', model)
val_metrics = evaluate('val', model)
test_metrics = evaluate('test', model)
log_to_file(log_file, str(f"\nMetrics = {test_metrics}"))

Metrics on train set:

Loss is 0.0101
MAE is 0.0586
RMSE is 0.1006
R² is 0.9907


Metrics on val set:

Loss is 0.0354
MAE is 0.0922
RMSE is 0.1884
R² is 0.9676


Metrics on test set:

Loss is 0.0145
MAE is 0.0896
RMSE is 0.1219
R² is 0.3846




In [80]:
label_model_files(old_name='hi', new_name='best')

Files have been renamed successfully.


In [238]:
model, optimizer, scheduler = load_model('best')

In [239]:
predictions_with_history_graphs(model, visualization_test_loader, path + "/Models/Global/LSTM/Graphs")
predictions_graphs(model, visualization_test_loader, path + "/Models/Global/LSTM/Graphs")
all_predictions_graphs(model, data, data_test, path + "/Models/Global/LSTM/Graphs")
all_predictions_with_gaps_graphs(model, data, data_test, path + "/Models/Global/LSTM/Graphs")
test_metrics = evaluate('test', model)

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'pandas._libs.tslibs.timestamps.Timestamp'>

should i adjust the data so that the lags are adjusted as if it hasnt seen it previously

In [147]:

dataset = pd.read_pickle(path + "/data/Volve/all_data.pkl")

# Ensure each entry in 'inputs' and 'targets' is a NumPy array
def to_numpy_array(x):
    return np.array(x, dtype=np.float32)

dataset['inputs'] = dataset['inputs'].apply(to_numpy_array)
dataset['targets'] = dataset['targets'].apply(to_numpy_array)

# Check the data structure
print(dataset.head())
print(dataset['inputs'].dtype)
print(dataset['targets'].dtype)

   well_name                                       inputs_dates  \
0       7405  [2014-04-22, 2014-04-23, 2014-04-24, 2014-04-2...   
1       7405  [2014-04-23, 2014-04-24, 2014-04-25, 2014-04-2...   
2       7405  [2014-04-24, 2014-04-25, 2014-04-26, 2014-04-2...   
3       7405  [2014-04-25, 2014-04-26, 2014-04-27, 2014-04-2...   
4       7405  [2014-04-26, 2014-04-27, 2014-04-28, 2014-04-2...   

                                       targets_dates     inputs_index  \
0  [2014-04-27, 2014-04-28, 2014-04-29, 2014-04-3...  [0, 1, 2, 3, 4]   
1  [2014-04-28, 2014-04-29, 2014-04-30, 2014-05-0...  [1, 2, 3, 4, 5]   
2  [2014-04-29, 2014-04-30, 2014-05-01, 2014-05-0...  [2, 3, 4, 5, 6]   
3  [2014-04-30, 2014-05-01, 2014-05-02, 2014-05-0...  [3, 4, 5, 6, 7]   
4  [2014-05-01, 2014-05-02, 2014-05-03, 2014-05-0...  [4, 5, 6, 7, 8]   

                                       targets_index  \
0  [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...   
1  [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17